# CNN Model for Digits Prediction

Building the CNN Model for Predicting the Digits of Sudoku.

In [1]:
import numpy as np 
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image

## Loading the Char74K Image Dataset

In [2]:
%cd /content/drive/MyDrive/Project-2

/content/drive/MyDrive/Project-2


In [3]:
path = '/content/drive/MyDrive/Project-2/Dataset'

In [4]:
data = os.listdir(path)

### Storing the Images and Labels in data_X and data_y

In [5]:
data_X = []     
data_y = [] 

data_classes = len(data)

for i in range (0,data_classes):
    data_list = os.listdir(path +"/"+str(i))
    for j in data_list:
        pic = cv2.imread(path +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic,(40,40))
        data_X.append(pic)
        data_y.append(i)

if len(data_X) == len(data_y):
    print("Total Dataponits = ", len(data_X))

# Labels and images
data_X = np.array(data_X)
data_y = np.array(data_y)

Total Dataponits =  10160


### Splitting the Train Data in Train Validation and Test Data

In [7]:
train_X, test_X, train_y, test_y = train_test_split(data_X,data_y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)

In [8]:
test_X.shape

(508, 40, 40, 3)

### Preprocessing and Reshaping the Images

In [9]:
def Prep(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img) 
    img = img/255 
    return img

In [10]:
train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))

In [11]:
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

In [12]:
train_X.shape

(7721, 40, 40, 1)

### Data Augmentation

In [13]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

### Converting the Labels into One-Hot Encoded Form

In [14]:
train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

## Building the CNN Model

In [15]:
model = Sequential()
model.add((Conv2D(60,(5,5),input_shape=(40, 40, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 40, 60)        1560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 60)        90060     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 30)        16230     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 30)        8130      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 30)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 30)        0

### Compile the Model

In [16]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Fit the model

In [17]:
model.fit(datagen.flow(train_X, train_y, batch_size=32), epochs = 30, validation_data = (valid_X, valid_y), verbose = 2, steps_per_epoch= 200)

Epoch 1/30
200/200 - 35s - loss: 0.9900 - accuracy: 0.6648 - val_loss: 0.2475 - val_accuracy: 0.9187
Epoch 2/30
200/200 - 3s - loss: 0.2857 - accuracy: 0.9100 - val_loss: 0.0696 - val_accuracy: 0.9741
Epoch 3/30
200/200 - 3s - loss: 0.2069 - accuracy: 0.9348 - val_loss: 0.0410 - val_accuracy: 0.9850
Epoch 4/30
200/200 - 3s - loss: 0.1538 - accuracy: 0.9566 - val_loss: 0.0290 - val_accuracy: 0.9902
Epoch 5/30
200/200 - 3s - loss: 0.1294 - accuracy: 0.9589 - val_loss: 0.0318 - val_accuracy: 0.9912
Epoch 6/30
200/200 - 3s - loss: 0.1146 - accuracy: 0.9616 - val_loss: 0.0175 - val_accuracy: 0.9953
Epoch 7/30
200/200 - 3s - loss: 0.1126 - accuracy: 0.9655 - val_loss: 0.0225 - val_accuracy: 0.9922
Epoch 8/30
200/200 - 3s - loss: 0.1060 - accuracy: 0.9690 - val_loss: 0.0204 - val_accuracy: 0.9938
Epoch 9/30
200/200 - 3s - loss: 0.0918 - accuracy: 0.9727 - val_loss: 0.0311 - val_accuracy: 0.9907
Epoch 10/30
200/200 - 3s - loss: 0.0867 - accuracy: 0.9745 - val_loss: 0.0319 - val_accuracy: 0.989

### Testing the Model's Score and Accuracy on the Test Data 

In [18]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Test Score =  0.02114090509712696
Test Accuracy = 0.9940944910049438


### Saving the Model

In [19]:
model.save('Model.h5')
model.save_weights('Model_weights.h5')